In [0]:
import pandas as pd
import numpy as np

# Data Cleaning

A great resource to have on hand (bookmark it!)

https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf


Creating a df by hand: 

In [0]:
some_data = [1, 2, 3, 4, 5] #numbers
some_more_data = ['a', 'b', 'c', 'd', 'e'] #letters
some_booleans = [True, False, True, True, True] #bools
df = pd.DataFrame({'numbers':some_data, 'letters':some_more_data, 'bools':some_booleans})
df

,numbers,letters,bools
0,1,a,True
1,2,b,False
2,3,c,True
3,4,d,True
4,5,e,True


### Changing data types



------






In [0]:
classes  = ['CS1111', 'PSYC1010', 'CS2150', 'ECON2010', 'SOC2010']
cf_ratings = ['4', 3.8, '0.2', 2, '4']
lou = pd.DataFrame({'courses':classes, 'ratings':cf_ratings})
lou

,courses,ratings
0,CS1111,4
1,PSYC1010,3.8
2,CS2150,0.2
3,ECON2010,2
4,SOC2010,4


In [0]:
lou['ratings_10'] = lou['ratings']*2
lou

,courses,ratings,ratings_10
0,CS1111,4,44
1,PSYC1010,3.8,7.6
2,CS2150,0.2,0.20.2
3,ECON2010,2,4
4,SOC2010,4,44


Drop that bad column that we just created!

Axes: Rows = 0, Columns = 1


In [0]:
lou = lou.drop('ratings_10', axis=1)

In [0]:
lou

,courses,ratings
0,CS1111,4
1,PSYC1010,3.8
2,CS2150,0.2
3,ECON2010,2
4,SOC2010,4


Convert all ratings to a float!

In [0]:
lou['ratings'] = lou['ratings'].apply(lambda x: float(x))

In [0]:
lou['ratings_10'] = lou.ratings * 2
lou

,courses,ratings,ratings_10
0,CS1111,4.0,8.0
1,PSYC1010,3.8,7.6
2,CS2150,0.2,0.4
3,ECON2010,2.0,4.0
4,SOC2010,4.0,8.0


In [0]:
lou['ratings'] = lou['ratings'].astype('str')
lou

,courses,ratings,ratings_10
0,CS1111,4.0,8.0
1,PSYC1010,3.8,7.6
2,CS2150,0.2,0.4
3,ECON2010,2.0,4.0
4,SOC2010,4.0,8.0


In [0]:
lou.ratings*2

0    4.04.0
1    3.83.8
2    0.20.2
3    2.02.0
4    4.04.0
Name: ratings, dtype: object

In [0]:
lou['ratings'] = lou['ratings'].astype('float')
lou.ratings*2

0    8.0
1    7.6
2    0.4
3    4.0
4    8.0
Name: ratings, dtype: float64

In [0]:
lou['ratings'] = lou['ratings'].astype('int')
lou

,courses,ratings,ratings_10
0,CS1111,4,8.0
1,PSYC1010,3,7.6
2,CS2150,0,0.4
3,ECON2010,2,4.0
4,SOC2010,4,8.0


## Handling missing values

You will often encounter missing data in your datasets. 

Take, for example, this dataframe with several NaN and None values:

In [0]:
students = ['Student_A', 'Student_B', 'Student_C', 'Student_D', np.nan, 'Student_F', 'Student_G']
years = [1, np.nan, 3, None, 4, np.nan, 1]
df = pd.DataFrame({'student':students,'year' :years})
df

,student,year
0,Student_A,1.0
1,Student_B,NaN
2,Student_C,3.0
3,Student_D,NaN
4,NaN,4.0
5,Student_F,NaN
6,Student_G,1.0


This will cause problems with many pandas and matplotlib methods.

### Identifying missing values


In [0]:
df.isna()

,student,year
0,False,False
1,False,True
2,False,False
3,False,True
4,True,False
5,False,True
6,False,False


In [0]:
df.isna().sum(axis=0) #Defaults to axis = 0

student    1
year       3
dtype: int64

In [0]:
df.isna().sum(axis=1)

0    0
1    1
2    0
3    1
4    1
5    1
6    0
dtype: int64

### 1. Dropping rows with missing values

By far the easiest way of dealing with missing data is just dropping rows that have missing values:

In [0]:
df_dropped = df.dropna()
df_dropped

,student,year
0,Student_A,1.0
2,Student_C,3.0
6,Student_G,1.0


You might also need to drop NAs only for a single column. In this case, we can pass an argument to .dropna()

In [0]:
df.dropna(subset=['year', 'student'])

,student,year
0,Student_A,1.0
2,Student_C,3.0
6,Student_G,1.0


 ### 2. Filling in missing values

We can fill in missing values in a variety of different ways. You can replace misisng values with a specific value (like the mean), forward-fill, back-fill, or use a variety of more advanced imputation methods such as KNN.

- Replacing missing values with a specific value:

In [0]:
df.fillna(0) # replace missing values with 0

,student,year
0,Student_A,1.0
1,Student_B,0.0
2,Student_C,3.0
3,Student_D,0.0
4,0,4.0
5,Student_F,0.0
6,Student_G,1.0


You can also use this to replace missing values with the mean of the values:

In [0]:
df_copy = df
df_copy['year'] = df['year'].fillna(df['year'].mean())
df_copy

,student,year
0,Student_A,1.00
1,Student_B,2.25
2,Student_C,3.00
3,Student_D,2.25
4,NaN,4.00
5,Student_F,2.25
6,Student_G,1.00


- another filling strategy that can be useful for timeseries data is back-fill and forward-fill in which we replace missing values with either the first value before or after the missing field.

In [0]:
df.fillna(method='ffill') # forward-fill

,student,year
0,Student_A,1.00
1,Student_B,2.25
2,Student_C,3.00
3,Student_D,2.25
4,Student_D,4.00
5,Student_F,2.25
6,Student_G,1.00


In [0]:
df.fillna(method='bfill') # back-fill

,student,year
0,Student_A,1.00
1,Student_B,2.25
2,Student_C,3.00
3,Student_D,2.25
4,Student_F,4.00
5,Student_F,2.25
6,Student_G,1.00


Don't use forward fill and back fill unless you're using time series data or data that has some other natural ordering! 



In [0]:
df['student'].fillna('Unknown Student', inplace = True)
df

,student,year
0,Student_A,1.00
1,Student_B,2.25
2,Student_C,3.00
3,Student_D,2.25
4,Unknown Student,4.00
5,Student_F,2.25
6,Student_G,1.00


### What fill method should I use?  

This is totally dependent on what question you're trying to answer!

#### Mike?

## String processing

Often, a dataset will contain string representations of data that could be really useful if you could find some way to extract it. 

In [0]:
import pandas as pd

In [0]:
people = ['Karmine Malhi', 'Ben Artuso', 'Mallory Lewis', 'Ishaan Dey', 'Brian Yu', 'Aaron Gu', 'Reilly Meinert'] 
node_class = ['Node B', 'node B', 'Convert B', 'Node b', 'Deploy A', 'Node A', 'Node C']

node_df = pd.DataFrame({'person':people, 'section':node_class})
node_df

,person,section
0,Karmine Malhi,Node B
1,Ben Artuso,node B
2,Mallory Lewis,Convert B
3,Ishaan Dey,Node b
4,Brian Yu,Deploy A
5,Aaron Gu,Node A
6,Reilly Meinert,Node C


In [0]:
people_list = node_df.person.str.split(' ')
people_list

0     [Karmine, Malhi]
1        [Ben, Artuso]
2     [Mallory, Lewis]
3        [Ishaan, Dey]
4          [Brian, Yu]
5          [Aaron, Gu]
6    [Reilly, Meinert]
Name: person, dtype: object

In [0]:
node_df['new_column'] = people_list.apply(lambda x: x[0])
node_df

,person,section,new_column
0,Karmine Malhi,Node B,Karmine
1,Ben Artuso,node B,Ben
2,Mallory Lewis,Convert B,Mallory
3,Ishaan Dey,Node b,Ishaan
4,Brian Yu,Deploy A,Brian
5,Aaron Gu,Node A,Aaron
6,Reilly Meinert,Node C,Reilly



It would be nice to separate first and last names out for easier searching. You may be familiar with the .split() function, which is used to split strings based on a space. 

In [0]:
"Karmine Malhi".split()

['Karmine', 'Malhi']

In [0]:
"Karmine Malhi".split('a')

['K', 'rmine M', 'lhi']

In [0]:
node_df.person 

0     Karmine Malhi
1        Ben Artuso
2     Mallory Lewis
3        Ishaan Dey
4          Brian Yu
5          Aaron Gu
6    Reilly Meinert
Name: person, dtype: object

In [0]:
node_df.person.split()

AttributeError: ignored

As you can see above, **string methods need some additional syntax to work on series**. 

We can add '.str' before any string operation to vectorize it across the whole series! 



In [0]:
node_df.person.str.split()

0     [Karmine, Malhi]
1        [Ben, Artuso]
2     [Mallory, Lewis]
3        [Ishaan, Dey]
4          [Brian, Yu]
5          [Aaron, Gu]
6    [Reilly, Meinert]
Name: person, dtype: object

QUESTION: How could we use this to create new columns for first and last name?

HINT: you will probably need to use '.apply(lambda x: \<yourfunctionhere\>)' after splitting!

In [0]:
node_df.person.apply(lambda x: )

0    False
1    False
2    False
3    False
4    False
5    False
6    False
Name: person, dtype: bool

#### Changing capitalization to better process text

Let's look at how many people of the 7 here are teaching each section! 

In [0]:
node_df.section.value_counts()

node B       1
Node C       1
Node b       1
Node A       1
Deploy A     1
Convert B    1
Node B       1
Name: section, dtype: int64

We can see here that **capitaliation presents a pretty big problem when working with a text data.**

An easy way to solve this is through converting all the text to a uniform case, again using .str before our string operations! 

In [0]:
"Ishaan Dey".upper()

'ISHAAN DEY'

In [0]:
node_df['section'] = node_df['section'].str.upper()
node_df

,person,section
0,Karmine Malhi,NODE B
1,Ben Artuso,NODE B
2,Mallory Lewis,CONVERT B
3,Ishaan Dey,NODE B
4,Brian Yu,DEPLOY A
5,Aaron Gu,NODE A
6,Reilly Meinert,NODE C


In [0]:
node_df['section'] = node_df['section'].str.lower()
node_df

,person,section
0,Karmine Malhi,node b
1,Ben Artuso,node b
2,Mallory Lewis,convert b
3,Ishaan Dey,node b
4,Brian Yu,deploy a
5,Aaron Gu,node a
6,Reilly Meinert,node c


In [0]:
node_df.section.value_counts()

node b       3
convert b    1
deploy a     1
node a       1
node c       1
Name: section, dtype: int64

**EXERCISE**: How many of the above people are in Node? (I know it's trivial and you can easily count the answer—that's so you know when you've coded the solution correctly)

In [0]:
#SOME SOLUTIONS: 
node_df 
node_df['big_section'] = node_df.section.apply(lambda x: x.split()[0])
node_df.section.str.startswith('node').sum()

5

## Date and time processing

pd.to_datetime is super powerful and flexible!

In [0]:
presidents = ['Washington' ,'Lincoln', 'Kennedy', 'Obama', 'Trump']
birthdays = ['Feb 27 1732', '2-12-1809', 'May 29th, 1917', '8 4 1961','06//14// //1946' ]

bdays = pd.DataFrame({'president': presidents, 'birthday': birthdays})
bdays

,president,birthday
0,Washington,Feb 27 1732
1,Lincoln,2-12-1809
2,Kennedy,"May 29th, 1917"
3,Obama,8 4 1961
4,Trump,06//14// //1946


In [0]:
bdays['birthday'] = pd.to_datetime(bdays['birthday'])
bdays

,president,birthday
0,Washington,1732-02-27
1,Lincoln,1809-02-12
2,Kennedy,1917-05-29
3,Obama,1961-08-04
4,Trump,1946-06-14


In [0]:
type(bdays.birthday[0])

pandas._libs.tslibs.timestamps.Timestamp

But, there are some formats even it won't recognize.

In [0]:
presidents = ['Washington' ,'Lincoln', 'Kennedy']
birthdays = ['2###27adjf1732', '2###12adjf1809', '5###05adjf1917']

bdays_messy = pd.DataFrame({'president': presidents, 'birthday': birthdays})
bdays_messy


,president,birthday
0,Washington,2###27adjf1732
1,Lincoln,2###12adjf1809
2,Kennedy,5###05adjf1917


In [0]:
bdays_messy['birthday'] = pd.to_datetime(bdays_messy['birthday'])
bdays_messy

ValueError: ignored

SO, you can specify the format yourself! Go to https://www.datacamp.com/community/tutorials/converting-strings-datetime-objects for some help

In [0]:
bdays_messy['birthday'] = pd.to_datetime(bdays_messy['birthday'], format = '%m###%dadjf%Y')
bdays_messy

,president,birthday
0,Washington,1732-02-27
1,Lincoln,1809-02-12
2,Kennedy,1917-05-05


### Using pandas datetime objects

In [0]:
washington = bdays.birthday[0]
print(washington)
washington.month

1732-02-27 00:00:00


2

In [0]:
washington.month_name()

'February'

In [0]:
washington.year

1732

In [0]:
washington.is_leap_year

True

In [0]:
washington.daysinmonth

29

### Make new columns from these datetime attributes 

Useful for feature engineering (see ya in week 6!)

In [0]:
bdays['month'] = bdays.birthday.apply(lambda x: x.month)
bdays['leap?'] = bdays.birthday.apply(lambda x: x.is_leap_year)
bdays['day'] = bdays.birthday.apply(lambda x: x.dayofweek)
bdays

,president,birthday,month,leap?,day
0,Washington,1732-02-27,2,True,2
1,Lincoln,1809-02-12,2,False,6
2,Kennedy,1917-05-29,5,False,1
3,Obama,1961-08-04,8,False,4
4,Trump,1946-06-14,6,False,4


In [0]:
bdays[['month', 'day'].apply(lambda x:)

president    0    Washington\n1       Lincoln\n2       Kenn...
birthday     0    1732-02-27 00:00:00\n1    1809-02-12 00:0...
month        0    2\n1    2\n2    5\n3    8\n4    6\nName: ...
leap?        0     True\n1    False\n2    False\n3    False...
day          0    2\n1    6\n2    1\n3    4\n4    4\nName: ...
dtype: object

### Datetime math

How much older was Washington than Lincoln?

In [0]:
diff = bdays.birthday[1] - bdays.birthday[0]
diff

Timedelta('28109 days 00:00:00')

In [0]:
diff.days / 365.25

76.95824777549623

What type is that object we just created? 

In [0]:
type(diff)

pandas._libs.tslibs.timedeltas.Timedelta

**Timedeltas** can be used to augment dates! It makes it very easy to change each date to be, say, 365 days later.

In [0]:
delta = pd.Timedelta(1, unit = 'y')
bdays

,president,birthday,month,leap?,day,augmented
0,Washington,1732-02-27,2,True,2,1732-03-12
1,Lincoln,1809-02-12,2,False,6,1809-02-26
2,Kennedy,1917-05-29,5,False,1,1917-06-12
3,Obama,1961-08-04,8,False,4,1961-08-18
4,Trump,1946-06-14,6,False,4,1946-06-28


In [0]:
bdays['augmented'] = bdays.birthday + delta
bdays

,president,birthday,month,leap?,day,augmented
0,Washington,1732-02-27,2,True,2,1733-02-26 05:49:12
1,Lincoln,1809-02-12,2,False,6,1810-02-12 05:49:12
2,Kennedy,1917-05-29,5,False,1,1918-05-29 05:49:12
3,Obama,1961-08-04,8,False,4,1962-08-04 05:49:12
4,Trump,1946-06-14,6,False,4,1947-06-14 05:49:12


### How can we apply this?

When you get data that includes time as a variable, it'll be in one of many possible formats, and not always consistent throughout the whole dataset. 


pd.to_datetime makes the process of cleaning these incredibly easy!


Once cleaned, we can look at specific attributes such as month, day, and year **to gain insight we wouldn't otherwise have been able to access.**

Seasonality!

## Merging DataFrames

Sometimes you have data from two different sources that you'd like to have in one data frame to analyze. We can do that with .concat() and .merge().

In [0]:
df1 = pd.DataFrame({'Dining': ['Castle', 'Newcomb', 'Chick-fil-A', 'Five Guys', 'Runk', 'Subway'], 'Rating': [4, 3, 5, 4, 4, 3]})
df1

,Dining,Rating
0,Castle,4
1,Newcomb,3
2,Chick-fil-A,5
3,Five Guys,4
4,Runk,4
5,Subway,3


In [0]:
df2 = pd.DataFrame({'Dining': ['Castle', 'Newcomb', 'Chick-fil-A', 'Five Guys', 'Runk', 'Subway'], 'Price': [8.5, 9.5, 6.5, 7.75, 9.5, 6]})
df2

,Dining,Price
0,Castle,8.50
1,Newcomb,9.50
2,Chick-fil-A,6.50
3,Five Guys,7.75
4,Runk,9.50
5,Subway,6.00


In [0]:
df3 = pd.DataFrame({'Dining': ['Castle', 'Newcomb', 'Chick-fil-A', 'Five Guys', 'Runk', 'Subway'],'Location': ['Old dorms', 'Central Grounds', 'Central Grounds', 'Central Grounds', 'Gooch-Dillard', 'Central Grounds']})
df3

,Dining,Location
0,Castle,Old dorms
1,Newcomb,Central Grounds
2,Chick-fil-A,Central Grounds
3,Five Guys,Central Grounds
4,Runk,Gooch-Dillard
5,Subway,Central Grounds


Now we want to put these into one data table, but not duplicate the "Dining" column. You can call .merge() on two data tables and specify the column to merge "on="

In [0]:
merged = pd.merge(df1, df2, on='Dining')
merged

,Dining,Rating,Price
0,Castle,4,8.50
1,Newcomb,3,9.50
2,Chick-fil-A,5,6.50
3,Five Guys,4,7.75
4,Runk,4,9.50
5,Subway,3,6.00


In [0]:
final_merged = pd.merge(merged, df3, on='Dining')
final_merged

,Dining,Rating,Price,Location
0,Castle,4,8.50,Old dorms
1,Newcomb,3,9.50,Central Grounds
2,Chick-fil-A,5,6.50,Central Grounds
3,Five Guys,4,7.75,Central Grounds
4,Runk,4,9.50,Gooch-Dillard
5,Subway,3,6.00,Central Grounds


Let's try .concat() to see if it does the same thing. It's a little different from .merge() because you have to pass in a list of dataframes that you want to concatenate.

In [0]:
pd.concat([df1, df2, df3])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Dining,Location,Price,Rating
0,Castle,NaN,NaN,4.0
1,Newcomb,NaN,NaN,3.0
2,Chick-fil-A,NaN,NaN,5.0
3,Five Guys,NaN,NaN,4.0
4,Runk,NaN,NaN,4.0
5,Subway,NaN,NaN,3.0
0,Castle,NaN,8.50,NaN
1,Newcomb,NaN,9.50,NaN
2,Chick-fil-A,NaN,6.50,NaN
3,Five Guys,NaN,7.75,NaN


So that didn't really work, because now we have duplicate rows with missing information. However, pd.concat() can be useful for a different case.

In [0]:
more_data = pd.DataFrame({'Dining': ["O'Hill", "Starbucks", "N2Go", "Burrito Theory"], 'Rating': [3, 4, 4, 3]})
more_data

,Dining,Rating
0,O'Hill,3
1,Starbucks,4
2,N2Go,4
3,Burrito Theory,3


In [0]:
df1

,Dining,Rating
0,Castle,4
1,Newcomb,3
2,Chick-fil-A,5
3,Five Guys,4
4,Runk,4
5,Subway,3


In [0]:
pd.concat([df1, more_data])

,Dining,Rating
0,Castle,4
1,Newcomb,3
2,Chick-fil-A,5
3,Five Guys,4
4,Runk,4
5,Subway,3
0,O'Hill,3
1,Starbucks,4
2,N2Go,4
3,Burrito Theory,3


Concat can actually add columns instead of rows too. Use the axis=1 argument. Here's a case where it might be useful.

In [0]:
more_info = pd.DataFrame({'Popularity': [8, 5, 10, 7, 8, 7], 'Hours': ["7:00-9:00", "7:00-8:00", "11:00-8:00", "11:00-8:00", "7:00-8:00", "11:00-8:00"]})
more_info

,Popularity,Hours
0,8,7:00-9:00
1,5,7:00-8:00
2,10,11:00-8:00
3,7,11:00-8:00
4,8,7:00-8:00
5,7,11:00-8:00


In [0]:
df = pd.concat([df1, df2, df3, more_info], axis=1)
df

,Dining,Rating,Dining,Price,Dining,Location,Popularity,Hours
0,Castle,4,Castle,8.50,Castle,Old dorms,8,7:00-9:00
1,Newcomb,3,Newcomb,9.50,Newcomb,Central Grounds,5,7:00-8:00
2,Chick-fil-A,5,Chick-fil-A,6.50,Chick-fil-A,Central Grounds,10,11:00-8:00
3,Five Guys,4,Five Guys,7.75,Five Guys,Central Grounds,7,11:00-8:00
4,Runk,4,Runk,9.50,Runk,Gooch-Dillard,8,7:00-8:00
5,Subway,3,Subway,6.00,Subway,Central Grounds,7,11:00-8:00


Note the difference between .concat(axis=1) and .merge(). We would use .concat() when there isn't a duplicate column, and .merge() when there is one.